In [1]:
import os
base_directory = '/workspaces/streaming-prj'
os.chdir(base_directory)

receive message

In [ ]:
import socket

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    print(f"Received message: {data.decode()}")


receive larger file

In [ ]:
import socket
import json

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Variable to hold the full message
received_data = []

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    if not data:
        break  # Exit on empty data (end of transmission)
    
    received_data.append(data.decode())  # Store received chunk
    print(f"Received chunk: {data.decode()[:50]}...")  # Show first 50 characters

# Reconstruct the full message
full_data = ''.join(received_data)

# Decode JSON data
try:
    json_message = json.loads(full_data)
    print(f"Received full message: {json_message['message'][:100]}...")  # Show first 100 characters
except json.JSONDecodeError:
    print("Failed to decode JSON data")

sock.close()


receive an image

In [ ]:
import socket

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Variable to hold the binary data
received_data = []
output_file_path = "image/received_image.jpg"  # Path to save the received file

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    if data == b'END':  # Check for the termination signal
        print("Received termination signal: 'END'. Stopping the transfer.")
        break  # Exit the loop when 'END' is received
    
    received_data.append(data)  # Store received chunk
    print(f"Received chunk of size {len(data)} bytes")

# Reconstruct the full binary file
with open(output_file_path, "wb") as output_file:
    for chunk in received_data:
        output_file.write(chunk)

print(f"File received and saved to {output_file_path}")
sock.close()


test audio

In [ ]:
import socket
import pygame
import io

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Variable to hold the binary data
received_data = []

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    if data == b'END':  # Check for the termination signal
        print("Received termination signal: 'END'. Stopping the transfer.")
        break  # Exit the loop when 'END' is received
    
    received_data.append(data)  # Store received chunk
    print(f"Received chunk of size {len(data)} bytes")

# Reconstruct the full audio data (WAV)
audio_data = b''.join(received_data)

# Initialize pygame mixer for audio playback
pygame.mixer.init()

# Use io.BytesIO to treat the received audio data as a file-like object
audio_wav = io.BytesIO(audio_data)

# Load the audio into pygame mixer
pygame.mixer.music.load(audio_wav)

# Play the audio
pygame.mixer.music.play()

# Wait for the audio to finish
while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)

print("Audio finished playing")
sock.close()


In [ ]:
import socket
import pygame
import io

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Variable to hold the binary data
received_data = io.BytesIO()  # Use a BytesIO buffer to store incoming data

# Initialize pygame mixer for audio playback
pygame.mixer.init()

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    if data == b'END':  # Check for the termination signal
        print("Received termination signal: 'END'. Stopping the transfer.")
        break  # Exit the loop when 'END' is received
    
    # Append received chunk to the buffer
    received_data.write(data)
    print(f"Received chunk of size {len(data)} bytes")

    # Attempt to play audio if we have enough data in the buffer
    if received_data.tell() >= 1024:  # Only start playing once we have at least 1KB of data
        # Rewind the buffer to the beginning
        received_data.seek(0)

        # Load the in-memory WAV data into pygame mixer
        pygame.mixer.music.load(received_data)

        # Play the audio
        pygame.mixer.music.play()

        # Wait for the audio to finish
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)
        
        # Clear the buffer after playing
        received_data.seek(0)
        received_data.truncate(0)

sock.close()


In [ ]:
import socket
import pygame
import io
from pydub import AudioSegment

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Variable to hold the binary data
received_data = io.BytesIO()  # Use a BytesIO buffer to store incoming data

# Receive the format information first
format_info, addr = sock.recvfrom(1024)  # Receive the format information (MP3/WAV)
format_info = format_info.decode()
print(f"Received format info: {format_info}")

# Initialize pygame mixer for audio playback
pygame.mixer.init()

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    if data == b'END':  # Check for the termination signal
        print("Received termination signal: 'END'. Stopping the transfer.")
        break  # Exit the loop when 'END' is received
    
    # Append received chunk to the buffer
    received_data.write(data)
    print(f"Received chunk of size {len(data)} bytes")

    # Attempt to play audio if we have enough data in the buffer
    if received_data.tell() >= 1024:  # Only start playing once we have at least 1KB of data
        # Rewind the buffer to the beginning
        received_data.seek(0)

        if format_info == "MP3":
            # Handle MP3 data (you might need an additional decoding step here)
            audio_data = received_data.read()  # You would decode the MP3 here
            # For simplicity, assume it can be played by pygame
            pygame.mixer.music.load(io.BytesIO(audio_data))
        elif format_info == "WAV":
            # Load the in-memory WAV data into pygame mixer directly
            pygame.mixer.music.load(received_data)

        # Play the audio
        pygame.mixer.music.play()

        # Wait for the audio to finish
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)
        
        # Clear the buffer after playing
        received_data.seek(0)
        received_data.truncate(0)

sock.close()


In [ ]:
import socket
import pygame
import io

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Variable to hold the binary data
received_data = io.BytesIO()  # Use a BytesIO buffer to store incoming data

# Receive the format information first
format_info, addr = sock.recvfrom(1024)  # Receive the format information (MP3/WAV)
format_info = format_info.decode()
print(f"Received format info: {format_info}")

# Initialize pygame mixer for audio playback
pygame.mixer.init()

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    if data == b'END':  # Check for the termination signal
        print("Received termination signal: 'END'. Stopping the transfer.")
        break  # Exit the loop when 'END' is received
    
    # Append received chunk to the buffer
    received_data.write(data)
    print(f"Received chunk of size {len(data)} bytes")

    # Attempt to play audio if we have enough data in the buffer
    if received_data.tell() >= 1024:  # Only start playing once we have at least 1KB of data
        # Rewind the buffer to the beginning
        received_data.seek(0)

        try:
            if format_info == "MP3":
                # Handle MP3 data (you might need an additional decoding step here)
                audio_data = received_data.read()  # You would decode the MP3 here
                # For simplicity, assume it can be played by pygame
                pygame.mixer.music.load(io.BytesIO(audio_data))
            elif format_info == "WAV":
                # Load the in-memory WAV data into pygame mixer directly
                pygame.mixer.music.load(received_data)

            # Play the audio
            pygame.mixer.music.play()

            # Wait for the audio to finish
            while pygame.mixer.music.get_busy():
                pygame.time.Clock().tick(10)

            # Clear the buffer after playing
            received_data.seek(0)
            received_data.truncate(0)

        except pygame.error as e:
            print(f"Error loading audio: {e}")
            received_data.seek(0)
            received_data.truncate(0)

sock.close()


In [ ]:
import socket
import pygame
import io

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Variable to hold the binary data
received_data = io.BytesIO()  # Use a BytesIO buffer to store incoming data

# Receive the format information first
format_info, addr = sock.recvfrom(1024)  # Receive the format information (MP3/WAV)
format_info = format_info.decode()
print(f"Received format info: {format_info}")

# Initialize pygame mixer for audio playback
pygame.mixer.init()

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    if data == b'END':  # Check for the termination signal
        print("Received termination signal: 'END'. Stopping the transfer.")
        break  # Exit the loop when 'END' is received
    
    # Append received chunk to the buffer
    received_data.write(data)
    print(f"Received chunk of size {len(data)} bytes")

    # Attempt to play audio if we have enough data in the buffer
    if received_data.tell() >= 1024:  # Only start playing once we have at least 1KB of data
        # Rewind the buffer to the beginning
        received_data.seek(0)

        try:
            if format_info == "MP3":
                # Handle MP3 data (you might need an additional decoding step here)
                audio_data = received_data.read()  # You would decode the MP3 here
                # For simplicity, assume it can be played by pygame
                pygame.mixer.music.load(io.BytesIO(audio_data))
            elif format_info == "WAV":
                # Load the in-memory WAV data into pygame mixer directly
                pygame.mixer.music.load(received_data)

            # Play the audio
            pygame.mixer.music.play()

            # Wait for the audio to finish
            while pygame.mixer.music.get_busy():
                pygame.time.Clock().tick(10)

            # Clear the buffer after playing
            received_data.seek(0)
            received_data.truncate(0)

        except pygame.error as e:
            print(f"Error loading audio: {e}")
            # Reset the buffer after an error
            received_data.seek(0)
            received_data.truncate(0)

sock.close()


In [ ]:
import socket
import pygame
import io

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5006  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Initialize pygame mixer for audio playback
pygame.mixer.init(frequency=44100, size=-16, channels=2)  # 44.1 kHz, 16-bit stereo PCM
print("Pygame mixer initialized.")

# Variable to hold the binary data
received_data = io.BytesIO()  # Use a BytesIO buffer to store incoming data

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    print(f"Received chunk of size {len(data)} bytes")
    
    if data == b'END':  # Check for the termination signal
        print("Received termination signal: 'END'. Stopping the transfer.")
        break  # Exit the loop when 'END' is received

    # Append received chunk to the buffer
    received_data.write(data)

    # Debugging: Print the buffer size and the state
    print(f"Buffer size: {received_data.tell()} bytes")
    
    # Attempt to play audio as data arrives
    if received_data.tell() >= 1024:  # Only start playing once we have at least 1KB of data
        # Rewind the buffer to the beginning
        received_data.seek(0)
        print("Loading and playing chunk...")

        try:
            # Use pygame's music module to load the raw PCM data for playback
            pygame.mixer.music.load(received_data)
            pygame.mixer.music.play()

            # Wait for the audio to finish
            while pygame.mixer.music.get_busy():
                pygame.time.Clock().tick(10)

            print("Audio playback finished.")

            # Clear the buffer after playback
            received_data.seek(0)
            received_data.truncate(0)

        except pygame.error as e:
            print(f"Error loading audio: {e}")
            # Reset the buffer after an error
            received_data.seek(0)
            received_data.truncate(0)

sock.close()


receive single chunk test

In [ ]:
import socket
import pygame
import io

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5006  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Initialize pygame mixer for audio playback
pygame.mixer.init(frequency=44100, size=-16, channels=2)  # 44.1 kHz, 16-bit stereo PCM
print("Pygame mixer initialized.")

# Variable to hold the binary data
received_data = io.BytesIO()  # Use a BytesIO buffer to store incoming data

while True:
    data, addr = sock.recvfrom(65507)  # Buffer size 1024 bytes
    print(f"Received chunk of size {len(data)} bytes")
    
    # if data == b'END':  # Check for the termination signal
    #     print("Received termination signal: 'END'. Stopping the transfer.")
    #     break  # Exit the loop when 'END' is received

    # Append received chunk to the buffer
    received_data.write(data)

    # Debugging: Print the buffer size and the state
    print(f"Buffer size: {received_data.tell()} bytes")
    
    # Play the chunk immediately as it arrives
    try:
        # Rewind the buffer to the beginning
        received_data.seek(0)
        print("Loading and playing chunk...")

        # Use pygame's Sound module to load raw PCM data for playback
        sound = pygame.mixer.Sound(received_data)  # Load the PCM data into pygame mixer
        sound.play()  # Play the audio

        # Wait for the audio to finish
        while pygame.mixer.get_busy():
            pygame.time.Clock().tick(10)

        print("Audio playback finished.")

        # # Clear the buffer after playback
        # received_data.seek(0)
        # received_data.truncate(0)

    except pygame.error as e:
        print(f"Error loading audio: {e}")
        # Reset the buffer after an error
        received_data.seek(0)
        received_data.truncate(0)

sock.close()


In [ ]:
import socket
import pygame
import io

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5006  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Initialize pygame mixer for audio playback
pygame.mixer.init(frequency=44100, size=-16, channels=2)  # 44.1 kHz, 16-bit stereo PCM
print("Pygame mixer initialized.")

# Variable to hold the binary data
received_data = io.BytesIO()  # Use a BytesIO buffer to store incoming data

while True:
    data, addr = sock.recvfrom(65507)  # Buffer size 65507 bytes (Max UDP packet size)
    print(f"Received chunk of size {len(data)} bytes")
    
    # Append received chunk to the buffer
    # received_data.write(data)

    # Debugging: Print the buffer size and the state
    # print(f"Buffer size: {received_data.tell()} bytes")
    
    # Play the chunk immediately as it arrives
    try:
        # Rewind the buffer to the beginning to read the data
        # received_data.seek(0)
        print("Loading and playing chunk...")

        # Use pygame's Sound module to load raw PCM data for playback
        sound = pygame.mixer.Sound(data)  # Load the PCM data into pygame mixer
        sound.play()  # Play the audio

        # Wait for the audio to finish
        while pygame.mixer.get_busy():
            pygame.time.Clock().tick(10)

        print("Audio playback finished.")

        # **Do not reset the buffer here**: Keep the buffer open for the next chunk.
        # received_data.seek(0)
        # received_data.truncate(0)  # We no longer reset the buffer here!

    except pygame.error as e:
        print(f"Error loading audio: {e}")
        # You can choose to reset the buffer if an error occurs:
        # received_data.seek(0)
        # received_data.truncate(0)

sock.close()


repeat calls for play

In [14]:
import socket
import pygame
import io
import queue
import threading

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5006  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Initialize pygame mixer for audio playback
pygame.mixer.init(frequency=44100, size=-16, channels=2)  # 44.1 kHz, 16-bit stereo PCM
print("Pygame mixer initialized.")

# Create a queue to buffer incoming audio chunks
audio_queue = queue.Queue()

# Variable to control playback status
is_playing = False

def play_audio_from_queue():
    global is_playing

    while True:
        # Get the next chunk from the queue
        if not audio_queue.empty():
            audio_data = audio_queue.get()

            try:
                # Load the audio chunk into pygame and play it
                sound = pygame.mixer.Sound(io.BytesIO(audio_data))
                sound.play()

                # Wait for the audio to finish
                while pygame.mixer.get_busy():
                    pygame.time.Clock().tick(10)

                print("Audio chunk finished playing.")
            
            except pygame.error as e:
                print(f"Error loading audio: {e}")

def receive_audio_data():
    global is_playing

    while True:
        data, addr = sock.recvfrom(65507)  # Buffer size 65507 bytes (Max UDP packet size)
        print(f"Received chunk of size {len(data)} bytes")

        # Add the received chunk to the queue
        audio_queue.put(data)
        print(f"Chunk added to queue. Queue size: {audio_queue.qsize()}")

# Start the playback thread
play_thread = threading.Thread(target=play_audio_from_queue, daemon=True)
play_thread.start()

# Start receiving audio data
receive_audio_data()

sock.close()


In [4]:
# Final version

import socket
import pygame
import io
import queue
import threading
import time

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5006  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Initialize pygame mixer for audio playback
pygame.mixer.init(frequency=44100, size=-16, channels=2)  # 44.1 kHz, 16-bit stereo PCM
print("Pygame mixer initialized.")

# Create a queue to buffer incoming audio chunks
audio_queue = queue.Queue()

# Variable to control playback status
is_playing = False

# Function to handle playback from the queue
def play_audio_from_queue():
    global is_playing

    while True:
        if not audio_queue.empty():
            # Get the next chunk from the queue
            audio_data = audio_queue.get()

            try:
                # Load and play the audio chunk
                sound = pygame.mixer.Sound(io.BytesIO(audio_data))
                sound.play()

                # Wait for the audio to finish
                while pygame.mixer.get_busy():
                    pygame.time.Clock().tick(10)

                print("Audio chunk finished playing.")

                # Add delay to match the chunk rate (250ms)
                # time.sleep(0.25)  # Synchronize with the 250ms chunk length

            except pygame.error as e:
                print(f"Error loading audio: {e}")

# Function to receive audio data and add to the queue
def receive_audio_data():
    while True:
        data, addr = sock.recvfrom(65507)  # Buffer size 65507 bytes (Max UDP packet size)
        print(f"Received chunk of size {len(data)} bytes")

        # Add the received chunk to the queue
        audio_queue.put(data)
        print(f"Chunk added to queue. Queue size: {audio_queue.qsize()}")

# Start the playback thread
play_thread = threading.Thread(target=play_audio_from_queue, daemon=True)
play_thread.start()

# Start receiving audio data
receive_audio_data()

sock.close()


Listening on 127.0.0.1:5006
Pygame mixer initialized.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Audio chunk finished playing.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Audio chunk finished playing.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Audio chunk finished playing.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Audio chunk finished playing.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Audio chunk finished playing.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 2
Audio chunk finished playing.
Received chunk of size 44144 bytes
Chunk added to queue.

KeyboardInterrupt: 

In [2]:

import socket
import pygame
import io
import queue
import threading
import time

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5006  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Initialize pygame mixer for audio playback
pygame.mixer.init(frequency=44100, size=-16, channels=2)  # 44.1 kHz, 16-bit stereo PCM
print("Pygame mixer initialized.")

# Create a queue to buffer incoming audio chunks
audio_queue = queue.Queue()

# Variable to control playback status
is_playing = False

# Function to handle playback from the queue
def play_audio_from_queue():
    global is_playing

    while True:
        if not audio_queue.empty():
            # Get the next chunk from the queue
            audio_data = audio_queue.get()

            try:
                # Load and play the audio chunk
                sound = pygame.mixer.Sound(io.BytesIO(audio_data))
                sound.play()

                # Wait for the audio to finish
                while pygame.mixer.get_busy():
                    pygame.time.Clock().tick(10)

                print("Audio chunk finished playing.")

                # Add delay to match the chunk rate (250ms)
                time.sleep(0.25)  # Synchronize with the 250ms chunk length

            except pygame.error as e:
                print(f"Error loading audio: {e}")

# Function to receive audio data and add to the queue
def receive_audio_data():
    while True:
        data, addr = sock.recvfrom(65507)  # Buffer size 65507 bytes (Max UDP packet size)
        print(f"Received chunk of size {len(data)} bytes")

        # Add the received chunk to the queue
        audio_queue.put(data)
        print(f"Chunk added to queue. Queue size: {audio_queue.qsize()}")

# Start the playback thread
play_thread = threading.Thread(target=play_audio_from_queue, daemon=True)
play_thread.start()

# Start receiving audio data
receive_audio_data()

sock.close()


pygame 2.6.1 (SDL 2.28.4, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
Listening on 127.0.0.1:5006
Pygame mixer initialized.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 1
Audio chunk finished playing.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 2
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 2
Audio chunk finished playing.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 3
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 3
Audio chunk finished playing.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 4
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 4
Audio chunk finished playing.
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 5
Received chunk of size 44144 bytes
Chunk added to queue. Queue size: 5
Rec


KeyboardInterrupt

